In [1]:
from langchain_community.document_loaders import CSVLoader
def load_data(file_path):
    loader = CSVLoader(file_path=file_path, encoding='utf-8')
    documents = loader.load()
    return documents

d:\Anaconda\envs\rag_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
documents = load_data("./law_data_3k.csv")

In [3]:
from langchain_text_splitters import CharacterTextSplitter
def text_split(documents):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=500, 
        chunk_overlap=50
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"分割完成！原始文档有 {len(documents)} 条，分割后变成了 {len(split_docs)} 个分块。")
    return split_docs

In [4]:
split_docs = text_split(documents)

分割完成！原始文档有 3013 条，分割后变成了 3725 个分块。


In [5]:
import os
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
def build_embeddingsdb(split_docs):
    os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
    model_name = "moka-ai/m3e-base"
    model_kwargs = {'device': 'cpu'} 
    encode_kwargs = {'normalize_embeddings': True} 
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    db = FAISS.from_documents(split_docs, embeddings)
    db.save_local("faiss_legal_db")

In [6]:
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [7]:
model_name = "moka-ai/m3e-base"
model_kwargs = {'device': 'cpu'} 
encode_kwargs = {'normalize_embeddings': True}

In [8]:
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

d:\Anaconda\envs\rag_env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\zqx79\.cache\huggingface\hub\models--moka-ai--m3e-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For bet

In [12]:
from tqdm import tqdm
db = FAISS.from_documents([split_docs[0]], embeddings)
for i in tqdm(range(1, len(split_docs))):
    db.add_documents([split_docs[i]])

100%|██████████| 3724/3724 [13:58<00:00,  4.44it/s]


In [13]:
db.save_local("faiss_legal_db")

In [14]:
query = "借款人去世，继承人是否应履行偿还义务？"

docs = db.similarity_search(query, k=3)

print(f"\n--- 针对问题：'{query}' 的检索结果 ---")
for i, doc in enumerate(docs):
    print(f"\n[结果 {i+1}]:")
    print(doc.page_content) 


--- 针对问题：'借款人去世，继承人是否应履行偿还义务？' 的检索结果 ---

[结果 1]:
data: 如果人死了，欠下的债家人要承担吗
债务人死亡之后由继承其遗产的人一并继承但是如果没有人继承其遗产或是其没有遗产的话债务随债务人死亡消除债权人无处主张

[结果 2]:
data: 请问，父亲生前欠下债务要还吗
你继承遗产的话，有可能需要

[结果 3]:
data: 人死了债务要家人还吗
人死了债务要家人还吗
死亡后债务一般有配偶承担连带责任，其他亲属如父母、子女不承担责任。一、死亡债务人配偶《中华人民共和国婚姻法》第十九条第三款规定：“夫妻对婚姻关系存续期间所得的财产约定归各自所有的，夫或妻一方对外所负的债务，第三人知道该约定的，以夫或妻一方所有的财产清偿”。根据反向解释规则，夫妻实行共同财产制或虽实行个人财产制但第三人不知道其约定的，均为夫妻共同债务。二、死亡债务人的继承人继承人承担偿还责任的法律规范基础是《中华人民共和国继承法》第三十三条：“继承遗产应当清偿被继承人依法应当缴纳的税款和债务，缴纳税款和清偿债务以他的遗产实际价值为限。超过遗产实际价值部分，继承人自愿偿还的不在此限。继承人放弃继承的，对被继承人依法应当缴纳的税款和债务可以不负偿还责任”
